# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção: (caso o site esteja preparado para evitar esse tipo de utilizaçao do selenium)

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [1]:
import pandas as pd

produtos_df = pd.read_excel('arquivos/buscas.xlsx')
display(produtos_df)
produtos_df.info()

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome            2 non-null      object
 1   Termos banidos  2 non-null      object
 2   Preço mínimo    2 non-null      int64 
 3   Preço máximo    2 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 192.0+ bytes


In [2]:
from selenium import webdriver # Comandar navegador controlado
from selenium.webdriver.chrome.service import Service # responsável por iniciar e parar o chromedriver
from webdriver_manager.chrome import ChromeDriverManager # Gerenciar arquvio ChromeDriver

# Instalar arquivo temporário ChromeDrive a partir da versão atual do Chrome
servico = Service(ChromeDriverManager().install())

# Abrir navegador com tela maximizada
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# Criar navegador
nav = webdriver.Chrome(service=servico, options=options)

In [3]:
# Entrar google shopping
nav.get('https://shopping.google.com/?nord=1')

In [4]:
from selenium.webdriver.common.by import By # Selecionar elementos
from selenium.webdriver.common.keys import Keys # Utilizar comandos do teclado

# Pesquisar produto
nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(produtos_df['Nome'][0])
nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(Keys.ENTER)



In [5]:
# Filtrar Valor

nav.find_element(By.XPATH, '//*[@id="rcnt"]/div[2]/div/div/div[3]/div[2]/div/div/div/span[6]/form/div[1]/div[1]/div/g-text-field/div/div[2]/input').send_keys(str(produtos_df['Preço mínimo'][0]))
nav.find_element(By.XPATH, '//*[@id="rcnt"]/div[2]/div/div/div[3]/div[2]/div/div/div/span[6]/form/div[1]/div[2]/div/g-text-field/div/div[2]/input').send_keys(str(produtos_df['Preço máximo'][0]))
nav.find_element(By.XPATH, '//*[@id="rcnt"]/div[2]/div/div/div[3]/div[2]/div/div/div/span[6]/form/div[1]/div[2]/button').click()

In [24]:
# Listando Valores Encontrados

# Listando elementos com informações do produto pesquisado
info_produto = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

# Listando termos banidos
termos_banidos = produtos_df['Termos banidos'][0].lower()
termos_banidos = termos_banidos.split(' ')

# Listando termos do nome do produto
termos_produto = produtos_df['Nome'][0].lower()
termos_produto = termos_produto.split(' ')

# Criando lista com os nome, preço e link do produto
resultado_pesquisa = []
for elemento in info_produto:
    
    # Selecionando elemento com nome do produto
    nome_produto = elemento.find_element(By.CLASS_NAME, 'tAxDx').text.lower()

    # Verificando termo banido
    produto_banido = False
    for termo in termos_banidos:
        if termo in nome_produto:
            produto_banido = True
    
    # Verificando se o nome do produto bate com o pesquisado
    produto_aceito = True
    for termo in termos_produto:
        if not termo in nome_produto:
            produto_aceito = False

    # Filtrando nome do produto
    if produto_aceito and not produto_banido:

        # Selecionando elemento com preço do produto
        preço = elemento.find_element(By.CLASS_NAME, 'a8Pemb').text

        # Selecionando elemento com link do produto
        elemento_link_filho = elemento.find_element(By.CLASS_NAME, 'aULzUe')
        elemento_link_pai = elemento_link_filho.find_element(By.XPATH, '..')
        link = elemento_link_pai.get_attribute('href')
        
        resultado_pesquisa.append((produtos_df['Nome'][0], preço, link))

resultado_pesquisa.sort(key= lambda x: x[1], reverse=False)
print(len(resultado_pesquisa), resultado_pesquisa)


24 [('iphone 12 64gb', 'R$ 3.024,00', 'https://www.google.com/url?url=https://doji.com.br/product/apple-iphone-12-branco-64gb-como-novo&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjUrfqV5N7_AhWur5UCHe77CPIQgOUECI8N&usg=AOvVaw34Tu-aqOmuyuiSqU4M5Ttl'), ('iphone 12 64gb', 'R$ 3.059,99', 'https://www.google.com/url?url=https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios-microsoft-365-personal-office-365-apps-1tb/p/2297528/te/ip11/%3F%26force%3D8%26seller_id%3Dmagazineluiza&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjUrfqV5N7_AhWur5UCHe77CPIQgOUECPgN&usg=AOvVaw3cegr68a9dsQyna5MRaLJh'), ('iphone 12 64gb', 'R$ 3.059,99', 'https://www.google.com/url?url=https://www.magazineluiza.com.br/iphone-11-apple-64gb-preto-61-12mp-ios-microsoft-365-personal-office-365-apps-1tb/p/2297527/te/ip11/%3F%26force%3D8%26seller_id%3Dmagazineluiza&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjUrfqV5N7_AhWur5UCHe77CPIQgOUECKoP&usg=AOvVaw0ZToFHrQAn1ZQKRLha38Ln'), ('iphone 12 64gb', 'R$ 3.059,99', 'https://www.google.com/url?url=ht

In [5]:
print(produtos_df['Preço mínimo'][0])

3000
